# Analyze images in batch with AI vision

Run the same AI prompt against multiple images automatically, without writing loops or managing API calls.

## Problem

You have a collection of images that all need the same analysis—like "Describe this image", "Is this product damaged?", or "What objects are visible?".

Writing a loop to call an API for each image is tedious and error-prone. You need to handle rate limits, retries, and track which images succeeded or failed.

| Image | Prompt | Expected Result |
|-------|--------|-----------------|
| product_1.jpg | "Is this damaged?" | "No visible damage" |
| product_2.jpg | "Is this damaged?" | "Scratch on surface" |
| product_3.jpg | "Is this damaged?" | "No visible damage" |

## Solution

**What's in this recipe:**
- Analyze multiple images with a single prompt using `openai.vision()`
- Get all results at once, stored in your table
- No loops or manual API calls

You add a computed column that applies `openai.vision()` to every image in your table. Pixeltable handles the API calls, retries, and result storage automatically.

When you insert new images, the analysis runs automatically—no extra code needed.

### Setup

In [ ]:
%pip install -qU pixeltable openai

import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

In [ ]:
import pixeltable as pxt
from pixeltable.functions import openai

### Load images

In [ ]:
# Create a fresh directory
pxt.drop_dir('vision_demo', force=True)
pxt.create_dir('vision_demo')

In [ ]:
t = pxt.create_table('vision_demo.images', {'image': pxt.Image})

In [ ]:
# Insert sample images
t.insert([
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000036.jpg'},
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000090.jpg'},
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000106.jpg'},
])

In [ ]:
# View loaded images
t.collect()

### Analyze images with AI

Add a computed column using `openai.vision()`. The prompt runs automatically on all images:

In [ ]:
# Define the prompt
PROMPT = "Describe this image in one sentence."

# Add computed column for AI analysis using openai.vision
t.add_computed_column(
    description=openai.vision(
        prompt=PROMPT,
        image=t.image,
        model='gpt-4o-mini'
    )
)

### View results

`openai.vision()` returns the text content directly:

In [ ]:
# View results: image alongside its AI-generated description
t.select(t.image, t.description).collect()

### New images are analyzed automatically

When you insert more images, the analysis runs without any extra code:

In [ ]:
# Insert a new image - analysis happens automatically
t.insert([{'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000139.jpg'}])

# View all results including the new image
t.select(t.image, t.description).collect()

## Explanation

**How it works:**

1. Add images to your table
2. Define a computed column with `openai.vision()`
3. Pixeltable executes the API call for each row automatically
4. Results are cached—rerunning won't re-call the API
5. New rows trigger automatic computation

**Changing the prompt:**

To use a different prompt, add a new computed column with `if_exists='replace'`:

```python
t.add_computed_column(
    description=openai.vision(prompt="New prompt", image=t.image, model='gpt-4o'),
    if_exists='replace'
)
```

**Using other providers:**

Replace `openai.vision` with:
- `anthropic.messages` for Claude
- `google.generate_content` for Gemini
- `together.chat_completions` for Together AI

## See also

- [Configure API keys](https://docs.pixeltable.com/howto/cookbooks/core/workflow-api-keys)
- [Working with OpenAI](https://docs.pixeltable.com/howto/providers/working-with-openai)

### New images are analyzed automatically

When you insert more images, the analysis runs without any extra code: